In [ ]:
import pandas as pd
import itertools
from itertools import chain
#Visualizations
import plotly.express as px
import plotly.graph_objects as go 
import matplotlib.cm as cm
import matplotlib.pyplot as plt


In [60]:
import networkx as nx
import pydot
import graphviz 
from IPython.display import display, Image
import pydotplus
from networkx.drawing.nx_pydot import write_dot
from IPython.display import IFrame

G = nx.complete_graph(5)
#source = nx.nx_pydot.to_pydot(G)
G.nodes[2]['color']='red'
G.nodes[2]['style']='filled'
G.edges[(0,1)]['color'] = 'grey'
G.edges[(0,1)]['style'] = 'filled'
G.edges[(0,1)]['penwidth'] = 10


#write_dot(G, "tree_1.dot")

#gvz=graphviz.Source(source)
# produce PDF
#gvz.view()
#print (gvz.source)
#display(gvz)

#pydot_graph = pydotplus.graph_from_dot_data(source)

#from IPython.display import Image

#Image(pydot_graph.create_png())



#with open("tree_1.dot") as f:
#    dot_graph = f.read()

# remove the display(...)

#graphviz.Source(dot_graph)

pos = nx.nx_pydot.pydot_layout(G, prog="sfdp")

#def view_pydot(pdot):
    #plt = Image(pdot.create_pdf())
    #display(plt)
    #pdot.write_pdf("test.pdf")
    #IFrame('test.pdf', width=600, height=300)
    
to_pdot = nx.drawing.nx_pydot.to_pydot
pdot = to_pdot(nx.complete_graph(5))
#plt = Image(pdot.create_pdf())
#display(plt)
#IFrame(plt, width=600, height=300)
#view_pydot(pdot)
#8,8 for png
# 
# 30,30 for pdf
pdot.set_size('"8,8!"')
pdot.write_jpeg("test.jpeg", prog='sfdp')
IFrame('test.', width=800, height=800)


In [47]:
help(pydot.Dot.write)

Help on function write in module pydot:

write(self, path, prog=None, format='raw', encoding=None)
    Writes a graph to a file.
    
    Given a filename 'path' it will open/create and truncate
    such file and write on it a representation of the graph
    defined by the dot object in the format specified by
    'format' and using the encoding specified by `encoding` for text.
    The format 'raw' is used to dump the string representation
    of the Dot object, without further processing.
    The output can be processed by any of graphviz tools, defined
    in 'prog', which defaults to 'dot'
    Returns True or False according to the success of the write
    operation.
    
    There's also the preferred possibility of using:
    
        write_'format'(path, prog='program')
    
    which are automatically defined for all the supported formats.
    [write_ps(), write_gif(), write_dia(), ...]
    
    The encoding is passed to `open` [1].
    
    [1] https://docs.python.org/3/librar

# Specifity values of kinase inhibitors

In [ ]:
import xlrd

df = pd.read_excel('/Users/cccccbbbbb/Downloads/kinase_inhibitor_selectivity.xlsm', sheet_name=None)
sheets = list(df.keys())
#DiscoverX data
dis = df[sheets[0]]
#Select inhibitors of interest
dis = dis[["kinase", "AKT...AZD5363","ERK...GDC-0994","MEK...Trametinib","PI3K...GDC-0941"]]


dis.loc[dis['kinase'] == 'MAP2K2']

dis.rename(columns={'AKT...AZD5363': 'AZD5363', 'ERK...GDC-0994': 'GDC0994','MEK...Trametinib':'Trametinib','PI3K...GDC-0941':'GDC0941'}, inplace=True)

dis

## Inferring relationship between kinase inhibitors specifity and community detection

In [ ]:
#Merge all dicts
all_communities = dict(ChainMap(*[PI3K_AKT_communities,MEK_ERK_communities]))
all_communities_nx = dict(ChainMap(*[PI3K_AKT_communities_nx,MEK_ERK_communities_nx]))
all_cs = dict(ChainMap(*[PI3K_AKT_cs,MEK_ERK_cs]))
targets_dict = dict(zip(all_communities.keys(),['MAP2K1','MAPK1_3','PIK3CA','AKT1_2']))
#Separate into communities other than main and loners
all_communities_group = {k:[x for x in all_communities[k] if len(x)>2 and targets_dict[k] not in x] for k in all_communities.keys()}
all_communities_single = {k:list(itertools.chain(*[x for x in all_communities[k] if len(x)<=2])) for k in all_communities.keys()}

#Make boxplot with concentrstion after treatment for each
all_cs

In [ ]:
rat = dis[dis['kinase'].isin(['PIK3CA','AKT1'])]

for tr in all_communities_group.keys():
    #tick_names
    kinase_com = [x[0] for x in all_communities_group[tr]]
    tick_names = [targets_dict[tr]] + kinase_com + ['communities \n ≤ 2 kinases']
    #values
    #target_val = dis[dis['kinase'].isin(list(all_communities_nx[tr][targets_dict[tr]]))][tr]
    #print(target_val)
    target_replace = {'MAPK1_3':['MAPK1','MAPK3'],'AKT1_2':['AKT1','AKT2']}
    target = list(itertools.chain(*[target_replace[x] if x in target_replace.keys() else [x] for x in list(all_communities_nx[tr][targets_dict[tr]])]))
    
    
target

In [ ]:
mean(nx.get_edge_attributes(net, 'weight').values())

mean(nx.get_node_attributes(net, 'community_strength_centrality').values())

In [ ]:
import seaborn as sns

for tr in all_communities_group.keys():
    #tick_names
    kinase_com = [x[0] for x in all_communities_group[tr]]
    tick_names = [targets_dict[tr]] + kinase_com + ['communities \n ≤ 2 kinases']
    #values
    #target_val = dis[dis['kinase'].isin(list(all_communities_nx[tr][targets_dict[tr]]))][tr]
    #print(target_val)
    target_replace = {'MAPK1_3':['MAPK1','MAPK3'],'AKT1_2':['AKT1','AKT2']}
    target = list(itertools.chain(*[target_replace[x] if x in target_replace.keys() else [x] for x in list(all_communities_nx[tr][targets_dict[tr]])]))
    target_values = dis[dis['kinase'].isin(target)][tr]
    group_values = [ dis[dis['kinase'].isin(list(all_communities_nx[tr][kin]))][tr] for kin in kinase_com]        
    #Add values for kinases with no communities
    values = [target_values] + group_values + [dis[dis['kinase'].isin(all_communities_single[tr])][tr] ]
    
    
    #Barplot of average edge weight instead
    fig = plt.figure()
    ax = fig.add_axes([0,0,1,1])
    langs = [targets_dict[tr]] + kinase_com
    students = [mean(nx.get_node_attributes(all_communities_nx[tr][kin], 'community_strength_centrality').values()) for kin in [targets_dict[tr]] + kinase_com]
    ax.bar(langs,students)
    plt.xticks(fontsize=17)
    #plt.show()
    
    
    fig, ax = plt.subplots()
    ax.boxplot(values)
    plt.xticks(range(1, len(tick_names)+1), tick_names, rotation=40, fontsize=12)
    sub_st = '$_{'
    sub_end = '}$'
    plt.title(tr + sub_st + '—' + sub_end + ' network')
    plt.show()
    
    
    #Scattered boxplot of CS instead 
    sns.set(style="whitegrid")
    #df = sns.load_dataset('iris')
    df = pd.DataFrame(all_cs[tr].items(), columns=['kinase', 'community strength centrality'])
    df["community"] = np.nan
    for com in [targets_dict[tr]] + kinase_com:
        df.loc[df['kinase'].isin(list(all_communities_nx[tr][com])),'community']= com
    df = df.dropna()
    #Change CSC to be normalised
    df['n_kinases'] = df['community'].apply(lambda x: len(list(all_communities_nx[tr][x])))
    df['community strength centrality'] = df['community strength centrality']/df['n_kinases']
    
    # Usual boxplot
    ax = sns.boxplot(x='community', y='community strength centrality', data=df, palette='pastel')
 
    # Add jitter with the swarmplot function
    ax = sns.swarmplot(x='community', y='community strength centrality', data=df, color="lightgrey", edgecolor='black', linewidth=0.5)
    plt.title(tr + sub_st + '—' + sub_end + ' network', fontweight='bold')
    plt.xlabel('community', fontweight='bold')
    plt.ylabel('community strength centrality', fontweight='bold')
    plt.savefig(f"output_visualizations/{tr}_other_com.pdf", format='pdf')
    #plt.show()


In [ ]:
list(all_communities_nx['GDC0994']['CSNK1E'])

In [ ]:
all_cs['GDC0994']['AKT1_2']

In [ ]:
ext='pdf'
treatment2 = 'GDC0994'
treatment = 'Trametinib'
kinasee = 'IRAK1'
net=all_communities_nx[treatment][kinasee]
font_threshold = 130/255
ext= 'pdf'

A = nx.nx_agraph.to_agraph(net)
#Map nodes weights to colors
max_color_lookup = list(nx.get_node_attributes(net, 'community_strength_centrality').values()) + list(nx.get_node_attributes(all_communities_nx[treatment2][kinasee], 'community_strength_centrality').values())
color_lookup = nx.get_node_attributes(net, 'community_strength_centrality')
min_value, *_, max_value = sorted(max_color_lookup)
print(max_color_lookup)
norm = mpl.colors.Normalize(vmin=min_value, vmax=max_value, clip=True)
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.GnBu) #Blues)
other_mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Oranges)
#Map edges weights to darker or lighter grey based on weight
max_ed_color_lookup = list(nx.get_edge_attributes(net, 'weight').values()) + list(nx.get_edge_attributes(all_communities_nx[treatment2][kinasee], 'weight').values())
ed_color_lookup = nx.get_edge_attributes(net, 'weight')
min_value, *_, max_value = sorted(max_ed_color_lookup)
norm = mpl.colors.Normalize(vmin=min_value, vmax=max_value, clip=True)
edges_mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Greys) #Blues)
for edge in ed_color_lookup.keys():
    ed = A.get_edge(edge[0],edge[1])
    ed.attr['color'] = mcolors.to_hex(edges_mapper.to_rgba(ed_color_lookup[edge]))
    ed.attr['penwidth'] = ed_color_lookup[edge]*3
#Draw nodes above edges
A.graph_attr["strict"] = False
A.graph_attr["overlap"] = False
A.graph_attr["splines"] = 'compound'
#Color by group
A.node_attr['style']='filled'
##Nodes present in this community but not the other(s)
for node in list(net):
    n = A.get_node(node)
    n.attr['fontsize'] = 22
    n.attr['fontname'] = "helvetica bold"
    #n.attr['fillcolor']="#9bd9f2"
    rgba = mapper.to_rgba(color_lookup[node])
    n.attr['fillcolor']= mcolors.to_hex(rgba)
    #Change font color to white if background is too dark
    if (rgba[0]*0.299 + rgba[1]*0.587 + rgba[2]*0.114) < font_threshold:
        n.attr['fontcolor']= '#ffffff'
        
##Nodes exclusive
unique = [x for x in list(net) if x not in list(all_communities_nx[treatment2][kinasee])]
for node in unique:
    n = A.get_node(node)
    n.attr['fontsize'] = 22
    n.attr['fontname'] = "helvetica bold"
    #n.attr['fillcolor']="#9bd9f2"
    rgba = other_mapper.to_rgba(color_lookup[node])
    n.attr['fillcolor']= mcolors.to_hex(rgba)
    #Change font color to white if background is too dark
    if (rgba[0]*0.299 + rgba[1]*0.587 + rgba[2]*0.114) < font_threshold:
        n.attr['fontcolor']= '#ffffff'

#Add colorbar legend for shared nodes
fig, ax = plt.subplots(1, 1)
fraction = 1
plt.colorbar(mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
ax.axis('off')
ax.set_title('absolute \n CSC of kinases', x=-0.3)
name = f"output_visualizations/{kinasee}.{treatment}_networkviz_static_shared_nodes_colorbar.{ext}"
plt.savefig(name, bbox_inches="tight",format=f"{ext}")
plt.show()

#Add colorbar legend for unique nodes
fig, ax = plt.subplots(1, 1)
fraction = 1
plt.colorbar(other_mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
ax.axis('off')
ax.set_title('absolute \n CSC of kinases', x=-0.3)
name = f"output_visualizations/{kinasee}.{treatment}_networkviz_static_unique_nodes_colorbar.{ext}"
plt.savefig(name, bbox_inches="tight",format=f"{ext}")
plt.show()
    
#Add colorbar legend for edges
fig, ax = plt.subplots(1, 1)
fraction = 1
plt.colorbar(edges_mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
ax.axis('off')
ax.set_title('absolute \n edges \n z-score (KSEA)', x=-0.3)
name = f"output_visualizations/{kinasee}.{treatment}_networkviz_static_edges_colorbar.{ext}"
plt.savefig(name, bbox_inches="tight",format=f"{ext}")
plt.show()

#Draw graph
A.layout()
title = f"output_visualizations/{treatment}_{kinasee}_networkviz_static"
A.draw(f"{title}.{ext}", prog='sfdp',format=f"{ext}")


In [ ]:
net=all_communities_nx['Trametinib']['CAMKK2']
kat = list(net) + ['AKT1','AKT2','MAPK1','MAPK3']

check = dis[dis['kinase'].isin(kat)]

check.sort_values(by=['Trametinib'])

## Comparisson of communities across network pairs

In [ ]:
#Get the appropriate color based on the weight
#[mapper.to_rgba(i) for i in color_lookup.values()]

#Trial graph
n=0
treatments = ['Trametinib', 'GDC0994']
t = treatments[n]
net = MEK_ERK_pos_communities_nx[t][com]
#Map nodes weights to colors
color_lookup = {k:v for v, k in enumerate(sorted(set(net.nodes())))}
low, *_, high = sorted(color_lookup.values())
norm = mpl.colors.Normalize(vmin=low, vmax=high, clip=True)
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.summer)

#pigraphviz instance from networkx graph
A = nx.nx_agraph.to_agraph(MEK_ERK_pos_communities_nx[treatments[n]][com])
n = A.get_node(node)
n.attr['penwidth']= 3
#n.attr['color']="#FFF300"
n.attr['color']="#4AF1F2"
#n.attr['fillcolor']="#9bd9f2"

#n.attr['fillcolor']= 
mcolors.to_hex(mapper.to_rgba(color_lookup['ARAF']))

In [ ]:
a = {('a','b'):1}

for key in a.keys():
    print(key[0])
    
from importlib import reload
plt=reload(plt)

In [ ]:
#https://manpages.debian.org/stretch/graphviz/dot.1.en.html

import pygraphviz
import os

com = 'MAPK14'
font_threshold = 130/255
ext= 'pdf'

#MEK_ERK
treatments = ['Trametinib', 'GDC0994']
shared_kin_nx = nx.intersection(MEK_ERK_pos_communities_nx['Trametinib'][com],MEK_ERK_pos_communities_nx['GDC0994'][com] )
nodes_inf = []
for n in range(len(treatments)):
    treat = treatments[n]
    net = MEK_ERK_pos_communities_nx[treat][com]
    #pigraphviz instance from networkx graph
    A = nx.nx_agraph.to_agraph(net)
    #Map nodes weights to colors
    color_lookup = nx.get_node_attributes(net, 'community_strength_centrality')
    min_value, *_, max_value = sorted(color_lookup.values())
    norm = mpl.colors.Normalize(vmin=min_value, vmax=max_value, clip=True)
    mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.GnBu) #Blues)
    other_mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Oranges)#YlOrBr)
    #Map edges weights to darker or lighter grey based on weight
    ed_color_lookup = nx.get_edge_attributes(net, 'weight')
    min_value, *_, max_value = sorted(ed_color_lookup.values())
    norm = mpl.colors.Normalize(vmin=min_value, vmax=max_value, clip=True)
    edges_mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Greys) #Blues)
    for edge in ed_color_lookup.keys():
        ed = A.get_edge(edge[0],edge[1])
        ed.attr['color'] = mcolors.to_hex(edges_mapper.to_rgba(ed_color_lookup[edge]))
        ed.attr['penwidth'] = ed_color_lookup[edge]*3
    #Draw nodes above edges
    A.graph_attr["strict"] = False
    A.graph_attr["overlap"] = False
    A.graph_attr["splines"] = 'compound'
    #Color by group
    A.node_attr['style']='filled'
    ##Nodes present in this community but not the other(s)
    unique_nodes = [node for node in list(net) if node not in shared_kin_nx]
    for node in unique_nodes:
        n = A.get_node(node)
        n.attr['fontsize'] = 22
        n.attr['fontname'] = "helvetica bold"
        #n.attr['fillcolor']="#9bd9f2"
        rgba = other_mapper.to_rgba(color_lookup[node])
        n.attr['fillcolor']= mcolors.to_hex(rgba)
        #Change font color to white if background is too dark
        if (rgba[0]*0.299 + rgba[1]*0.587 + rgba[2]*0.114) < font_threshold:
            n.attr['fontcolor']= '#ffffff'
    ##Nodes present in both communities being compared
    for node in list(shared_kin_nx):
        n = A.get_node(node)
        n.attr['fontsize'] = 22
        n.attr['fontname'] = "helvetica bold"
        #n.attr['fillcolor']="#9bd9f2"
        rgba = mapper.to_rgba(color_lookup[node])
        n.attr['fillcolor']= mcolors.to_hex(rgba)
        #Change font color to white if background is too dark
        if (rgba[0]*0.299 + rgba[1]*0.587 + rgba[2]*0.114) < font_threshold:
            n.attr['fontcolor']= '#ffffff'
    #Highlight MEK/ERK
    for node in ['ARAF','MAP2K1','MAPK1_3']:
        if node in list(net):
            n = A.get_node(node)
            n.attr['shape']='diamond'
            n.attr['width']=2
            n.attr['height']=2
            n.attr['fixedsize']='true' 
            #n.attr['penwidth']= 3
            #n.attr['color']="#FFF300"
            #n.attr['color']="#4AF1F2"
            #n.attr['fillcolor']="#9bd9f2"
            #n.attr['fillcolor']= mcolors.to_hex(mapper.to_rgba(color_lookup[node]))
    #Highlight PIK3CA/AKT
    for node in ['PIK3CA','AKT1_2','MTOR']:
        if node in list(net):
            n = A.get_node(node)
            n.attr['shape']='square'
            n.attr['width']=1.2
            n.attr['fixedsize']='true'
            #n.attr['penwidth']= 3
            #n.attr['color']="#FF6EFF"
            #n.attr['fillcolor']="#9bd9f2"
            #n.attr['fillcolor']= mcolors.to_hex(mapper.to_rgba(color_lookup[node]))
    
    ##Multi-panel colorbar
    #Boxes
    title_nodes = 'CSC of kinases (nodes)'
    title_shapes = 'canonical \n pathways'
    title_edges = 'edges z-score \n (KSEA)'
    #Suboxes
    title_shared = 'present \n in both SCs'
    title_unique = 'exclusive \n to this SC'
    
    #Add colorbar legend for unique nodes 
    fig, ax = plt.subplots(1, 1)
    fraction = 1
    plt.colorbar(other_mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
    ax.axis('off')
    ax.set_title('CSC of kinases \n exclusive \n to this SC', x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_unique_nodes_colorbar.{ext}"
    plt.savefig(name, bbox_inches="tight",format=f"{ext}")
    plt.show()
    
    #Add colorbar legend for shared nodes
    fig, ax = plt.subplots(1, 1)
    fraction = 1
    plt.colorbar(mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
    ax.axis('off')
    ax.set_title('CSC of kinases \n present \n in both SCs', x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_shared_nodes_colorbar.{ext}"
    plt.savefig(name, bbox_inches="tight",format=f"{ext}")
    plt.show()
    
    #Add shapes legend
    fig, ax = plt.subplots(1, 1)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_shared_nodes_colorbar.pdf"
    a = ax.scatter([0], [0], marker='s', facecolors='none', edgecolors='black', label= 'PI3K/AKT/mTOR')
    b = ax.scatter([0], [0], marker='D', facecolors='none', edgecolors='black', label= 'ARAF/MAP2K1/MAPK1_3' )
    # adjust the figure size as necessary
    fig_leg = plt.figure(figsize=(1,1))
    ax_leg = fig_leg.add_subplot(111)
    # add the legend from the previous axes
    ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', title='canonical pathways')
    # hide the axes frame and the x/y labels
    ax_leg.axis('off')
    ax.set_title(title_shapes, x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_pathways_legend.{ext}"
    fig_leg.savefig(name, bbox_inches="tight",format=f"{ext}")
    fig_leg.show()

    #Add colorbar legend for edges
    fig, ax = plt.subplots(1, 1)
    fraction = 1
    plt.colorbar(edges_mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
    ax.axis('off')
    ax.set_title(title_edges, x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_edges_colorbar.{ext}"
    plt.savefig(name, bbox_inches="tight",format=f"{ext}")
    plt.show()
    
    #Draw graph
    A.layout()
    title = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static.pdf"
    A.draw(title, prog='sfdp')

treatments = list(PI3K_AKT_pos_communities_nx.keys())
shared_kin_nx = nx.intersection(PI3K_AKT_pos_communities_nx[treatments[0]][com],PI3K_AKT_pos_communities_nx[treatments[1]][com] )
nodes_inf = []
for n in range(len(treatments)):
    treat = treatments[n]
    net = PI3K_AKT_pos_communities_nx[treat][com]
    #pigraphviz instance from networkx graph
    A = nx.nx_agraph.to_agraph(PI3K_AKT_pos_communities_nx[treatments[n]][com])
    #Map nodes weights to colors
    color_lookup = nx.get_node_attributes(net, 'community_strength_centrality')
    min_value, *_, max_value = sorted(color_lookup.values())
    norm = mpl.colors.Normalize(vmin=min_value, vmax=max_value, clip=True)
    mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.GnBu) #Blues)
    other_mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Oranges)#YlOrBr)
    #Map edges weights to darker or lighter grey based on weight
    ed_color_lookup = nx.get_edge_attributes(net, 'weight')
    min_value, *_, max_value = sorted(ed_color_lookup.values())
    norm = mpl.colors.Normalize(vmin=min_value, vmax=max_value, clip=True)
    edges_mapper = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Greys) #Blues)
    for edge in ed_color_lookup.keys():
        ed = A.get_edge(edge[0],edge[1])
        ed.attr['color'] = mcolors.to_hex(edges_mapper.to_rgba(ed_color_lookup[edge]))
        ed.attr['penwidth'] = ed_color_lookup[edge]*4
    #Draw nodes above edges
    A.graph_attr["strict"] = False
    A.graph_attr["overlap"] = False
    A.graph_attr["splines"] = 'compound'
    #Color by group
    A.node_attr['style']='filled'
    ##Nodes present in this community but not the other(s)
    unique_nodes = [node for node in list(net) if node not in shared_kin_nx]
    for node in unique_nodes:
        n = A.get_node(node)
        n.attr['fontsize'] = 22
        n.attr['fontname'] = "helvetica bold"
        #n.attr['fillcolor']="#9bd9f2"
        rgba = other_mapper.to_rgba(color_lookup[node])
        n.attr['fillcolor']= mcolors.to_hex(rgba)
        #Change font color to white if background is too dark
        if (rgba[0]*0.299 + rgba[1]*0.587 + rgba[2]*0.114) < font_threshold:
            n.attr['fontcolor']= '#ffffff'
    ##Nodes present in both communities being compared
    for node in list(shared_kin_nx):
        n = A.get_node(node)
        n.attr['fontsize'] = 22
        n.attr['fontname'] = "helvetica bold"
        #n.attr['fillcolor']="#9bd9f2"
        rgba = mapper.to_rgba(color_lookup[node])
        n.attr['fillcolor']= mcolors.to_hex(rgba)
        #Change font color to white if background is too dark
        if (rgba[0]*0.299 + rgba[1]*0.587 + rgba[2]*0.114) < font_threshold:
            n.attr['fontcolor']= '#ffffff'
    #Highlight MEK/ERK
    for node in ['ARAF','MAP2K1','MAPK1_3']:
        if node in list(net):
            n = A.get_node(node)
            n.attr['shape']='square'
            n.attr['width']=1.5
            n.attr['fixedsize']='true' 
            #n.attr['penwidth']= 3
            #n.attr['color']="#FFF300"
            #n.attr['color']="#4AF1F2"
            #n.attr['fillcolor']="#9bd9f2"
            #n.attr['fillcolor']= mcolors.to_hex(mapper.to_rgba(color_lookup[node]))
    #Highlight PIK3CA/AKT
    for node in ['PIK3CA','AKT1_2','MTOR']:
        if node in list(net):
            n = A.get_node(node)
            n.attr['shape']='diamond'
            n.attr['width']=1.5
            n.attr['height']=2
            n.attr['fixedsize']='true'
            #n.attr['penwidth']= 3
            #n.attr['color']="#FF6EFF"
            #n.attr['fillcolor']="#9bd9f2"
            #n.attr['fillcolor']= mcolors.to_hex(mapper.to_rgba(color_lookup[node]))
    
    
    ##Multi-panel colorbar
    
    #Boxes
    title_nodes = 'CSC of kinases (nodes)'
    title_shapes = 'canonical \n pathways'
    title_edges = 'edges z-score \n (KSEA)'
    #Suboxes
    title_shared = 'kinases \n present \n in both SCs'
    title_unique = 'kinases \n exclusive \n to this SC'
    
    #Add colorbar legend for unique nodes 
    fig, ax = plt.subplots(1, 1)
    fraction = 1
    plt.colorbar(other_mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
    ax.axis('off')
    ax.set_title('CSC of kinases \n exclusive \n to this SC', x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_unique_nodes_colorbar.{ext}"
    plt.savefig(name, bbox_inches="tight",format=f"{ext}")
    plt.show()
    
    #Add colorbar legend for shared nodes
    fig, ax = plt.subplots(1, 1)
    fraction = 1
    plt.colorbar(mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
    ax.axis('off')
    ax.set_title('CSC of kinases \n present \n in both SCs', x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_shared_nodes_colorbar.{ext}"
    plt.savefig(name, bbox_inches="tight",format=f"{ext}")
    plt.show()
    
    #Add shapes legend
    fig, ax = plt.subplots(1, 1)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_shared_nodes_colorbar.{ext}"
    a = ax.scatter([0], [0], marker='D', facecolors='none', edgecolors='black', label= 'PI3K/AKT/mTOR')
    b = ax.scatter([0], [0], marker='s', facecolors='none', edgecolors='black', label= 'ARAF/MAP2K1/MAPK1_3' )
    # adjust the figure size as necessary
    fig_leg = plt.figure(figsize=(1,1))
    ax_leg = fig_leg.add_subplot(111)
    # add the legend from the previous axes
    ax_leg.legend(*ax.get_legend_handles_labels(), loc='center', title='canonical pathways')
    # hide the axes frame and the x/y labels
    ax_leg.axis('off')
    ax.set_title(title_shapes, x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_pathways_legend.{ext}"
    fig_leg.savefig(name, bbox_inches="tight",format=f"{ext}")
    fig_leg.show()

    #Add colorbar legend for edges
    fig, ax = plt.subplots(1, 1)
    fraction = 1
    plt.colorbar(edges_mapper, ax=ax, pad=.05, extend='both', fraction=fraction)
    ax.axis('off')
    ax.set_title(title_edges, x=-0.3)
    name = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static_edges_colorbar.{ext}"
    plt.savefig(name, bbox_inches="tight",format=f"{ext}")
    plt.show()
    
    
    #Draw graph
    A.layout()
    title = f"output_visualizations/MAPK14.{treat}_pos_networkviz_static"
    A.draw(f"{title}.{ext}", prog='sfdp',format=f"{ext}")
    #os.system(f"pdftoppm -png -r 300  {title}.{ext} {title}")

In [ ]:
PIL.features.pilinfo()

In [ ]:
##Make multi-panel figure from existing figures
import PIL
from PIL import Image
#from pdf2image import convert_from_path
from PyPDF2 import PdfReader

fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)

ax1_im = Image.open("output_visualizations/MAPK14.AZD5363_pos_networkviz_static-1.png")

ax1.imshow(ax1_im)

plt.savefig('output_visualizations/multiplot.pdf')

In [ ]:
#Arrange communities by the kinases with the highest CSC
##Join all communities into one dictionnary for ease
all_com_pos_nx = {**MEK_ERK_pos_communities_nx, **PI3K_AKT_pos_communities_nx}
all_com_pos = {**MEK_ERK_pos_communities, **PI3K_AKT_pos_communities}
##Store kinases that were grouped alone
all_com_loners = {treat:[x[0] for x in all_communities[treat] if len(x)==1] for treat in all_com_pos.keys() }

##Make dictionnary with the kinases with the highest CSC for each community
##Remove communities of one kinase at the same time
##Note that it is returning the kinase with the highest number of edges, not with the highest CSC
all_com_pos_by_max_sc = { treat:{max(com.degree(weight='weight'))[0]:com for com in set(all_com_pos_nx[treat].values()) if len(list(com))>1 } for treat in all_com_pos_nx.keys()}

#Join pairs of networks by highest CSC kinase
MEK_ERK_com_pos_sc = join_with(list, [all_com_pos_by_max_sc['Trametinib'],all_com_pos_by_max_sc['GDC0994']])
PI3K_AKT_com_pos_sc = join_with(list, [all_com_pos_by_max_sc['GDC0941'],all_com_pos_by_max_sc['AZD5363']])
all_com_paired = [{key:value for key,value in x.items() if len(x[key])==2} for x in [MEK_ERK_com_pos_sc, PI3K_AKT_com_pos_sc] ]

#Where the highest CSC kinase does not match, find what the match is (they share at least two nodes)
unmatched = [ {k:v for k,v in x.items() if len(x[k])==1} for x in [MEK_ERK_com_pos_sc, PI3K_AKT_com_pos_sc] ]
kin_pairs = [ list(itertools.combinations(list(unmatched[0].keys()),2)) , list(itertools.combinations(list(unmatched[1].keys()),2)) ]
all_com_cs_unmatched_paired = [ {pair:[unmatched[n][pair[0]][0], unmatched[n][pair[1]][0]] for pair in kin_pairs[n] if len(list(nx.intersection(unmatched[n][pair[0]][0], unmatched[n][pair[1]][0]))) >= 2} for n in [0,1]]
##Since only one new match ...
##Remove TNK2 and PIK3CB from 2nd one
PI3K_updated = {**PI3K_AKT_com_pos_sc, **all_com_cs_unmatched_paired[1]}
del PI3K_updated['PIK3CB']
del PI3K_updated['TNK2']
all_com_results = [MEK_ERK_com_pos_sc, PI3K_updated]

#Calculate average edge weight in each community


yes_com = [ list(all_com_pos_nx[x]['YES1']) for x in all_com_pos_nx.keys() ]
yes_com_shared = set(yes_com[0]).intersection(*yes_com)

#set(p[0]).intersection(*p)

yes_com_shared

nx.get_edge_attributes(all_com_pos_nx['Trametinib']['YES1'], 'weight')

{k:v for k,v in PI3K_AKT_pos_cs['AZD5363'].items() if k in list(MEK_ERK_pos_communities_nx['Trametinib']['MAPK14'])}



In [ ]:
PI3K_AKT_pos_cs['GDC0941']['PIK3CB']

In [ ]:
PI3K_AKT_com_kin = { key: list(x[0] for x in value if len(x)>1) for key, value in PI3K_AKT_pos_communities.items() }
MEK_ERK_com_kin = { key: list(x[0] for x in value if len(x)>1) for key, value in MEK_ERK_pos_communities.items() }

PI3K_AKT_com_kin

In [ ]:
#Calculate average edge weight for each community
##Save identifying kinase for each of the communities with more than one member
PI3K_AKT_com_kin = { key: list(x[0] for x in value if len(x)>1) for key, value in PI3K_AKT_pos_communities.items() }
MEK_ERK_com_kin = { key: list(x[0] for x in value if len(x)>1) for key, value in MEK_ERK_pos_communities.items() }
#all_com_kin = PI3K_AKT_com_kin + MEK_ERK_com_kin
#treatments_order = list(PI3K_AKT_pos_communities.keys()) + list(MEK_ERK_pos_communities.keys())

##Calculate the average edge weight for each
MEK_ERK_pos_ave_weight = 
PI3K_AKT_pos_ave_weight = 

#Identify kinase with highest strength centrality across pair 
##Sum SC between the pair

##Find highest kinase for each

In [ ]:
PI3K_AKT_pos_cs
R = G.copy()
R.remove_nodes_from(n for n in G if n not in H)
R.remove_edges_from(e for e in G.edges if e not in H.edges)

In [ ]:
nx.intersection(MEK_ERK_pos_communities_nx['Trametinib']['MAPK14'],MEK_ERK_pos_communities_nx['GDC0994']['MAPK14'] )

In [ ]:
edges = nx.get_edge_attributes(MEK_ERK_pos_communities_nx[treatments[n]]['MAPK14'], 'weight')
from_ed = [x[0] for x in edges.keys() ]
to_ed = [x[1] for x in edges.keys() ]
edges_list = list(zip(from_ed,to_ed, edges.values(),edges.values() ))
edges_df = pd.DataFrame(edges_list, columns=['from','to','z-score','weight'])

edges_df

# Comparisson of MTOR community in negative and positive edges

In [ ]:
neg_pos_target_communities = [target_communities2, pos_target_communities2[0]]

neg_pos_target_communities

In [ ]:
[x1,y1,z1] = cs_calculation(treat_int=[0,1], communities=neg_pos_target_communities, treatments=community_names)

# HEATMAP
fig = go.Figure(data=go.Heatmap(
                z=z1,
                x=x1,
                y=y1,
                hoverongaps = False))
fig.show()

# TTK in colorectal cancer cell lines

In [ ]:
networkfile = pd.read_excel('input/edges cell lines and primary breast.xlsm', sheet_name='edges cell lines and primary br')

networkfile = networkfile[["edge","difference hct116","difference dld1"]]

# Convert first column (kinase interactions/pairs) of the document to a list
edges_list= networkfile[networkfile.columns[0]].to_list()
# Convert the list into a list of tuples, each tuple being one kinase pair/edge (since this is the format needed to 
# add edges onto pyvis.network (network visualization package))
edges_list2 = pd.DataFrame({'col':[tuple(x.split('.',1)) for x in edges_list]})
addr = ['from','to']
networkfile = networkfile.join(edges_list2.col.apply(lambda loc: pd.Series(loc, index=addr)))

#Select rows that are pik3ca/akt12/mtor edges
networkfile = networkfile[ (networkfile['from'].isin(['TTK'])) | (networkfile['to'].isin(['TTK']))]

#Kinases
kinases = list(set(itertools.chain(*[networkfile['from'],networkfile['to']])))
kinases.remove('TTK')
kin_dict = {kin:set(itertools.chain(*[networkfile[ (networkfile['from']==kin) | (networkfile['to']==kin) ][["from","to"]] ][0].values.tolist())) for kin in ['TTK']}
values = [ [networkfile[( (networkfile['from']==kin) & (networkfile['to']==x) )|( (networkfile['from']==x) & (networkfile['to']==kin) )]['difference hct116'] if (kin in kin_dict[x]) and (len(networkfile[( (networkfile['from']==kin) & (networkfile['to']==x) )|( (networkfile['from']==x) & (networkfile['to']==kin) )]['difference hct116'])==1) else None for kin in kinases] for x in ['TTK'] ]
values1 = [ [float(x) if x is not None else None for x in listi ] for listi in values ]
values = [ [networkfile[( (networkfile['from']==kin) & (networkfile['to']==x) )|( (networkfile['from']==x) & (networkfile['to']==kin) )]['difference dld1'] if (kin in kin_dict[x]) and (len(networkfile[( (networkfile['from']==kin) & (networkfile['to']==x) )|( (networkfile['from']==x) & (networkfile['to']==kin) )]['difference dld1'])==1) else None for kin in kinases] for x in ['TTK'] ]
values2 = [ [float(x) if x is not None else None for x in listi ] for listi in values ]

#kin='PIK3CA'
#x='AKT1_2'
#float(networkfile[( (networkfile['from']==kin) & (networkfile['to']==x) )|( (networkfile['from']==x) & (networkfile['to']==kin) )]['difference hct116'])

#pandas.core.series.Series
#plt.savefig('output_visualizations/hdct116_heatmap.pdf')

values3 = values1 + values2

values3


In [ ]:
# HEATMAP
fig = go.Figure(data=go.Heatmap(
                z=values3,
                x=kinases,
                y= ['TTK <br>HCT116 cell line', 'TTK <br>DLD1 cell line'],
                yaxis='y2',
                #title= 'edges z-scores in HCT116 cells \n with PIK3CA mutations vs wild-type',
                hoverongaps = False))

fig.update_layout(xaxis=dict(domain=[0.15,1]),
                    yaxis2=dict(anchor='free', position=0, side='right'),
                     title= 'z-score difference of TTK edges in cells with PIK3CA mutations vs wild-type'
                 )

fig.update_layout(
    title={
        'y':0.86,
        'x':0.55,
        'xanchor': 'center'},
    font={
        'size':13
    }
)

fig.update_yaxes(tickfont_size=14)
fig.update_xaxes(tickfont_size=14)



# Interactive network visualization

In [ ]:
# input: edges_list is a list corresponding to a treatment in the dataset, it's contents being the kinase interactions
# in said treatment with a z_score <0 and their z_scores,in the shape of tuples, i.e. a network
# kinase_groups is a list of the groups in which we want to classify the kinases in the network
# kg_colors is a list of the colors in which each kinase groups should appear in the visualization
# treatment is the header of the column from which the z-scores for edges_list was extracted (i.e. name of the treatment)
# output: A visualization of the network constructed from the edges list, in HTML format
def network_visualization(edges_list, kinases, kg_colors, treatment):    
    #Initialize a pyvis.network instance
    networkgraph = Network(height='1000px',width='1000px', heading='')
    
    #Add each community of nodes with their assigned color
    for n in range(len(kinases)):
        for x in kinases[n]:
            networkgraph.add_node(x, value=15000000000, title=x, label=x, color=kg_colors[n], mass=23)
    
    #Add the edges
    networkgraph.add_edges(edges_list)
    
    #Make empty dictionary in which to store the degree of each kinase
    degree_graphs = {}
    #Save list of nodes to extract information such as kinases degree centrality
    kinases_map = networkgraph.get_adj_list()

    for node in networkgraph.nodes:
        #Add degree to each kinase in the empty dictionary
        degree_graphs[node["title"]] = len(kinases_map[node["id"]]) 
        #Add box showing degree + to which nodes each node is connected, which appears when the user hovers over a node on the network graph
        node["title"] += " connected to "+ str(len(kinases_map[node["id"]])) +" kinases:<br>" + "<br>".join(kinases_map[node["id"]]) 
        kinase = len(kinases_map[node["id"]])
        #Change the nodes to be bigger the higher their degree is
        if  0 <=  kinase <= 19 :
            node["value"] = 100000

        if  20 <=  kinase <= 29 :
            node["value"] = 1000000000

        if  30 <=  kinase <= 39 :
            node["value"] = 2000000000

        if  40 <=  kinase <= 49:
            node["value"] = 10000000000
    
        if  50 <=  kinase <= 59:
            node["value"] = 20000000000

        if 60 <=  kinase <= 69:
            node["value"] = 30000000000
        
        if  70 <=  kinase:
            node["value"] = 40000000000
    
    #Settings regarding the appearance of the network
    networkgraph.set_options("""
    var options = {
    "nodes": {
    "borderWidth": 1.5,
        "font": {
        "size": 80,
        "face": "verdana"
        }
      },
      "edges": {
        "color": {
          "color": "rgba(192,187,223,1)",
          "inherit": true
        },
        "font": {
          "strokeWidth": 6
        },
        "hoverWidth": 3.2,
        "smooth": false
      },
      "physics": {
        "barnesHut": {
          "springLength": 1040
        },
        "minVelocity": 0.75
      }
    }
    """)
    
    #Return visualization as HTML file
    final_network = treatment + '[-]'
    networkgraph.show('output_visualizations/network_visualization{}.html'.format(final_network))


## 2.2 - Network visualizations of networks trametinib(MAP2K1)[-], GDC0994(MAPK1_3), GDC0941(PI3K)[-], and AZD5363(AKT)[-]

In [ ]:
for n in range(len(target_communities)):
    networkgraph = network_visualization(edges_list=negative_edges[n],kinases=kinase_groups[n],kg_colors=["#fcbb8b", "#857be3", "#baeeff"], treatment=treatments[n])

## 2.3 - Network visualization of intersection between the selected communities of  trametinib(MAP2K1)[-], GDC0941(PI3K)[-] and AZD5363(AKT)[-]

In [ ]:
#Kinases present in the selected community of network trametinib(MAP2K1)[-], but not in the selected communities of 
#networks GDC0941(PIK3CA)[-] and AZD5363(AKT1_2)[-]
#MAP2K1only = [x for x in communities[0] if x not in communities[1] and x not in communities[2]]
#Kinases present accross the selected communities of networks trametinib(MAP2K1)[-], GDC0941(PIK3CA)[-] and AZD5363(AKT1_2)[-]
intersection = [x for x in target_communities[0] if x in target_communities[1] and x in target_communities[2]]
#Kinases present accross the selected communities of networks trametinib(MAP2K1)[-] and GDC0941(PIK3CA)[-], but not in the
#selected community of network AZD5363(AKT)[-]
MAP2K1_PIK3CA = [x for x in target_communities[0] if x in target_communities[1] and x not in target_communities[2]]
#Kinases present accross the selected communities of networks AZD5363(AKT)[-] and GDC0941(PIK3CA)[-] but not trametinib(MAP2K1)[-]
#and kinases present only in the selected communities of GDC0941(PIK3CA)[-] and AZD5363(AKT1_2)[-] networks
#therest = [[x for x in communities[3] if x in communities[2] and x not in communities[0]],[x for x in communities[3] if x not in communities[2] and x not in communities[0]], [x for x in communities[2] if x not in communities[3] and x not in communities[0]]]

kinase_groups = MAP2K1_PIK3CA, intersection
kinases = list(itertools.chain(*kinase_groups))
#Store kinase interactions across the three networks in which the two kinases are in any of the selected communities
#edges = negative_edges[0] + negative_edges[1] + negative_edges[2]
#edges = [(a,b,0) for a,b,c in edges if a in kinases and b in kinases]
edges = [ [(a,b) for a,b,c in negative_edges[n]] for n in [0,1,2] ]
edges = set(edges[0]).intersection(edges[1], edges[2])
edges = [(a,b) for a,b in edges if a in kinases and b in kinases]

#Intersection of communities visualized as a network
networkgraph= network_visualization(edges_list=edges,kinases=kinase_groups,kg_colors=["#fcbb8b", "#857be3", "#baeeff","green"], treatment='intersection_MAP2K1_PIK3CA_AKT')


# Return target names from column headers

In [ ]:
## Return targets (if they are present in the treatment name)
targets = []
#csv with KSEA kinase names and all uniprot aliases
kinase_gene_ids = pd.read_csv('input/kinase_gene_names.csv')
#Return targets for each treatment
for treatment in headers:
    ## Return targets (if they are present in the treatment name)
    #Separate targets from treatment
    sep_target_treat = treatment.split('.')[0:-1]
    #Separate kinase name from isoform numbers
    target = [re.findall('\d+|\D+', x) if x[-1].isdigit() else x for x in sep_target_treat  ]
    if len(target) > 1:
        target = [item for sublist in target for item in sublist]
        #Add kinase name with each of the isoform numbers
        name_range = list(range(0,len(target)))
        name_ids = [n for n in name_range if target[n].isnumeric()==False]    
        name_ids.append(len(target)-1)
        name_range_split = [ name_range[name_ids[n-1]:(name_ids[n]+1)] for n in list(range(1,len(name_ids)))]
        #Paste each kinase with its isoforms numbers
        target = [ [target[x[0]]+target[y] for y in x[1:len(x)]] for x in name_range_split]
        target = [item for sublist in target for item in sublist]
    #Replace with KSEA kinase names
    for n in list(range(0,len(target))):
        kin = target[n].upper()
        df1 = kinase_gene_ids[kinase_gene_ids.apply(lambda row: row.astype(str).str.contains( kin+'_', case=False).any(), axis=1)]
        if len(df1) == 0:
            df1 = kinase_gene_ids[kinase_gene_ids.apply(lambda row: row.astype(str).str.contains(kin, case=False).any(), axis=1)]
        target[n] = list(dict.fromkeys(df1["node"]))
        
    #Add to list of all treatments targets
    target = [item for sublist in target for item in sublist]
    targets.append(target)